# Reglas de Asociación

### Objetivo

<div style="text-align: justify"><b>
Usa la librería mlextend que nos permita solucionar todos los problemas relacionados con las reglas de asociación. Para ello tendremos que usar el algoritmo Apriori y una serie de métodos para obtener la distinta información que este genera.
</b></div>

###### Librerías del código

In [2]:
%reset -f
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import matplotlib.pyplot as plt
from mlxtend.plotting import stacked_barplot
from mlxtend.plotting import checkerboard_plot

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

#### Metodos 

Metodos descritos en el apartado 1 que seran probados en los demás apartados

Este metodo nos devuelve los itemsets frecuentes según el valor k.

In [3]:
def kitemsets(df, min_soporte):
    print("----------------------- K - ITEMSETS -----------------------------------")
    aprior = apriori (df, min_support=min_soporte, use_colnames=True, max_len = math.inf)
    data = pd.DataFrame(aprior["support"])
    data["itemsets"] = aprior["itemsets"]
    data["K-itemsets"] = len(data["itemsets"][0])
    for x in range (0, len(data)):
        data["K-itemsets"][x] = len(data["itemsets"][x])

    k = 1
    datos = []
    for ki in range (0, len(data)):
        if data["K-itemsets"][ki] == k:
            datos.append([data["itemsets"][ki], data["support"][ki]])
        else:
            kdatos = pd.DataFrame(datos)
            print("K - itemset siendo k = ", k)
            display(kdatos)
            print("---------------------------------------")
            k = data["K-itemsets"][ki]
            datos = []
            datos.append([data["itemsets"][ki], data["support"][ki]])
    #repetimos aqui para mostrar la tabla del ultimo k-itemset encontrado
    kdatos = pd.DataFrame(datos, columns = ["itemsets", "support"])
    print("K - itemset siendo k = ", k)
    display(kdatos)
    print("-------------------------------------------------------------------------")
    
    #print(data)
    
    return data

Se muestran con este método todas las reglas de asociación que se producen independientemente del valor de confianza que posean. Se realiza utilizando la funcion "associaton_rules" de la libreria mlxtend y poniendo el parámetro "min_threshold" a cero que es el umbral minimo de la metrica que hemos seleccionado, en este caso se trata de la confianza, es decir, el mínimo umbral de confianza es 0.

In [4]:
def mostrar_todas_reglas(frequents_items):
    print("----------------- MOSTRAR TODAS LAS REGLAS DE ASOCIACIÓN --------------------------")
    rules = association_rules(frequents_items, metric="confidence", min_threshold=0)
    display(rules)
    grafico_1(rules)
    grafico_2(rules)
    print("-------------------------------------------------------------------------")
    return rules

Método que muestra aquellas reglas que sean de alta confianza. Principalmente el funcionamento es similar al anterior método. Sin embargo, en este caso, se cambia el minimo umbral de confianza segun se desee.

In [5]:
def reglas_alta_confianza(frequents_items):
    print("----------------- REGLAS DE MAYOR CONFIANZA --------------------------")
    print("Consideraremos que tienen alta confianza cuando superan el 50% de confianza (0.5)")
    rules2 = association_rules(frequents_items, metric="confidence", min_threshold=0.5)
    if rules2.empty == True:
        print("No hay reglas de asociación con la suficiente confianza")
    else:
        display(rules2)
        grafico_1(rules2)
        grafico_2(rules2)
    print("-------------------------------------------------------------------------")

Con este metodo se puede realizar una busque mediane la introducción por teclado del antecedente que se desee consutar. Con ello se consigue ver de manera especifica las reglas que tienen dicho antecedente.

In [6]:
def busqueda_por_antecesor(rules):
#     rules = association_rules(frequents_items, metric="confidence", min_threshold=0)
#     display(rules)
    print("----------------- BUSQUEDA POR ANTECESOR --------------------------")
    antecesores_validos = [rules["antecedents"][0]]
    for i in range (0, len(rules)):
        for j in range (0, len(antecesores_validos)):
            if antecesores_validos[j] != rules["antecedents"][i] and j == len(antecesores_validos)-1:
                antecesores_validos.append(rules["antecedents"][i])
            elif antecesores_validos[j] == rules["antecedents"][i]:
                break
    print("Lista de antecesores validos")
    for y in range(0, len(antecesores_validos)):
        indix = str(antecesores_validos[y])
        indix = indix.replace("frozenset({'","")
        indix = indix.replace("'})","")
        indix = indix.replace("', '",", ")
        antecesores_validos[y] = indix
    display(antecesores_validos)          
    print("Introduzca antecesor valido, es decir, el nombre del producto que se quiere utilizar como antecesor")
    antecesor = input()
    antecesor = antecesor.split(', ', 1 )
    antecesor2 = antecesor[0]
    if len(antecesor) == 1:
        antecedente = rules[rules['antecedents'] == {antecesor2}]
    else:
        for x in range(1, len(antecesor)):
            antecesor2 = (antecesor2, antecesor[x])
        print(antecesor2)
        antecedente = rules[rules['antecedents'] == frozenset(antecesor2)]
    
    
    display(antecedente)
    grafico_2(antecedente)
    print("-------------------------------------------------------------------------")

Método que tras introducir un umbral de confianza (con valores entre 0.0 y 1.0) muestra las reglas que tienen ese valor de confianza o mayor

In [7]:
def umbral_min_reglas_x_confianza(rules, confianza):
    print("----------------- BUSQUEDA POR UMBRAL DE CONFIANZA --------------------------")
    print("Introducir valor minimo de confianza para buscar. Valores entre 0 y 1. Ejemplo: 0.5")
    #confianza = input()
    cumple = rules[ (rules['confidence'] >= float(confianza))]
    display(cumple)
    grafico_1(cumple)
    grafico_2(cumple)
    print("-------------------------------------------------------------------------")

Se muestra mediante un gráfico de barras, se muestran los valores de confianza, soporte y Lift. Para ello se necesita que el dataframe tenga dichas columnas

In [8]:
def grafico_1(df): 
    df = transformar_a_str(df)
    df2 = pd.DataFrame(df, columns = ["confidence", "support", "lift"])
    df = unir_variables(df)
    index = []
    for x in range(0, len(df["rules"])):
        index.append(df["rules"][x])
    
    plt = stacked_barplot(df2, rotation=90, legend_loc='best', labels = index)
    plt.show()

Se genera un gráfico de dispersión con el que se observará el valor de confianza de cada una de las reglas segun sean sus antecedentes y cinsecuentes.

In [9]:
def grafico_2(df):
    df = transformar_a_str(df)
    #df2 = pd.DataFrame(df, columns = ["confidence", "support", "lift"])
    plt.figure(figsize=(20,10))
    plt.scatter(df['antecedents'],
                df["consequents"],
                df['confidence']*1000, #Multiplicamos el valor por 1000 para ver los resultados de manera más clara, aunque el valor real sera el obtenido 
                c = df['confidence'])
    #.figsize=(len(df["consequents"])*2,len(df['antecedents'])*2)
    plt.suptitle('Confianza de las distintas reglas')
    plt.ylabel('Consequents')
    plt.xlabel('Antecedents')
    plt.colorbar()
    plt.grid()
    plt.show()

In [10]:
def transformar_a_str(df):
    df = df.reset_index()
    indice = []
    indice2 = []
    for x in range(0, len(df["antecedents"])):
        indix = str(df["antecedents"][x])
        indix = indix.replace("frozenset({'","")
        indix = indix.replace("'})","")
        indix = indix.replace("', '",", ")
        indice.append(indix)

        indix = str(df["consequents"][x])
        indix = indix.replace("frozenset({'","")
        indix = indix.replace("'})","")
        indix = indix.replace("', '",", ")
        indice2.append(indix)

    df["antecedents"] = indice
    df["consequents"] = indice2
    
    #df = df.reset_index()
    return df

In [11]:
def unir_variables(df):
    df2 = pd.DataFrame(df, columns = ["confidence", "support", "lift"])
    df2["rules"] = df["antecedents"] + ", " + df["consequents"]
    return df2

Método mediante el cual, tras recibir un dataframe y una lista con los valores que utilizaremos como soporte y umbral, realiza los metodos descritos en el enunciado con el fin de obtener los items frecuentes, las reglas tanto de alta confianza como todas, la creacion de los diferentes graficos y los metodos de busqueda por antecesor y según el valor de umbral de confianza.

In [12]:
def obtener_datos(df, lista_soporte_umbral):
    for x in range(0, len(lista_soporte_umbral)):
        frequents_items = kitemsets(df, lista_soporte_umbral[x][0])
        rules = mostrar_todas_reglas(frequents_items)
        reglas_alta_confianza(frequents_items)
        plt.show()
        busqueda_por_antecesor(rules)
        umbral_min_reglas_x_confianza(rules, lista_soporte_umbral[x][1])
        print("##############################################################################################")
        wait = input("Para realizar el siguiente test pulse intro")
        clear_output()
        

## Apartado 2. Dataset Compras.csv

Se abre el archivo csv y se procede a su transformación para que pueda ser procesado por la librearía de manera correcta. 
Tras ello se ha creado una lista que contiene diferentes valores de soporte y umbral que se utilizarán a la hora de obtener los datos para su interpretación.

In [16]:
file = open('radio.csv') 
lines = file.readlines()
for x in range (0, len(lines)):
    lines[x] = lines[x].replace("\n", "")
    lines[x] = lines[x].split(",")
    

te = TransactionEncoder()
te_ary = te.fit(lines).transform(lines)
df = pd.DataFrame(te_ary, columns=te.columns_)
display(df.head(10))
wait = input("Pulse intro para Continuar")
clear_output()
#Lista de pruebas de soporte y umbral
lista_soporte_umbral = ([[0.03, 0.4],
                         [0.04, 0.3],
                         [0.02, 0.5]])

obtener_datos(df, lista_soporte_umbral)

FileNotFoundError: [Errno 2] No such file or directory: 'radio.csv'

### Análisis Resultados

##### Soporte = 0.03; Umbral = 0.4

Con estos valores podemos observar en primer lugar que solo podeos conseguir hata k=2. Tambien resulta interesante observar que de todas las reglas posibles creadas (37) , la confianza nunca supera el valor del 50%.
También es remarcable el hecho de que solo dos reglas tienen un nivel de lift menor que 1, lo que implica que no son reglas buenas, ambas reglas son: whole milk, soda y soda,whole milk.
En el gráfico de dispersión podemos observar que las reglas que más confianza poseen son aquellas cuyo antecedente son root vegetables y que aquellas cuyo antecedente es whole milk tienen una confianza bastante baja.

##### Soporte = 0.04; Umbral = 0.4

En este caso, al igual que en el caso anterior, el maximo valor k de los k-itemset que se puede obtener es de k=2.
Cabe destacar que con estos valores, debido al incremento del soporte minimo, se reduce el número de reglas creadas de 37 a 17, pero al igual que ocurría en el caso anterior, ninguna de las reglas alcanza un nivel de confianza de por lo menos un 50%. 
De igual modo, en el gráfico de dispersión, las reglas que generan una mayor confianza son aquellas cuyo antecedente son las root vegetables, y las que menos, las de whole milk. Sin embargo, cuando el consecuente se trata de whole milk los valores de la regla se ven incrementados.

##### Soporte = 0.05; Umbral = 0.3

Al igual que ha ocurrido en los casos anteriores, el valor máximo de k posible es k=2. 
Se ha producido una reducción más grande del número de reglas encontradas, en este caso solo exitiendo 5. Todas ellas con un valor de lift mayor de 1, ya que las reglas wholw milk,soda y soda,whole milk han desaparecido con estos valores.
En este caso, destaca tambien que ya no hay reglas cuyo antecedente sea root vegetables, las cuales eran siempre las que proporcionaban un mayor valor de confianza, aunque podemos observar que ahora la que mayor confianza tiene es la de other vegetables. Viendo los tres ejemplos, podemos entonces afirmar que las que tienen que ver con verduras (root vegetables y other vegetables) como antecedente, son las que nos han arrojado un mayor nivel de confianza. Y que aquellas cullo antecedente es la whole milk, siempre nos ha dado como las de peor nivel de confianza.

Resulta curioso observar que en todos los casos whole milk como antecedente no suele generar reglas con una cnfianza alta en comparación con el resto, dando como resultado las más bajas. Sin embargo, cuando no es antecedente sino consecuete, los valores mejoran. Es decir que la leche de ese tipo, suele ser un producto que se compra después del resto, los clientes que compran leche en primer lugar no suelen comprar algo más después pero al contrario sí.

## Apartado 3. Dataset Cancer de mama

Se abre el archivo csv y se procede a su transformación para que pueda ser procesado por la librearía de manera correcta. 
Tras ello se ha creado una lista que contiene diferentes valores de soporte y umbral que se utilizarán a la hora de obtener los datos para su interpretación.

In [ ]:
file = open("Lab2 Recursos/Datos/Cancer Pecho/cancer.csv") 
lines = file.readlines()
for x in range (0, len(lines)):
    lines[x] = lines[x].replace("\n", "")
    lines[x] = lines[x].split(",")  


te = TransactionEncoder()
te_ary = te.fit(lines).transform(lines)
df = pd.DataFrame(te_ary, columns=te.columns_)
display(df.head(10))
wait = input("Pulse intro para Continuar")
clear_output()
#Lista de pruebas de soporte y umbral
lista_soporte_umbral = ([[0.5, 0.7],
                         [0.6, 0.6],
                         [0.4, 0.5]])

obtener_datos(df, lista_soporte_umbral)

### Análisis Resultados

##### Soporte = 0.5; Umbral = 0.7

Podemos observar que el máximo valor posible para k es K=3. Se han generado 12 reglas en total, todas ellas poseen un valor de lift superior a 1 y una gran confianza. 
Del gráfico de dispersión destaca principalmente que los mayores valores de confianza se obtienen cuando el consecuente es "no", que es el valor de la etiqueta node-caps. De ellos, la mayoria tienen como antecedente no-recurrence-events, siendo estas reglas las que nos presentan unos valores ligeramente mayores al resto.

##### Soporte = 0.6; Umbral = 0.6

En esta configuración se ha visto reducido el valor de K a k=2. De igual forma se ha reducido el numero de reglas hasta 4. Al igual que ocurría con la configuración anterior, tenemos valores de confianza considerablemente grandes, siendo el mínimo 0.73. 
Podemos observar tambien, en el gráfico de dispersión, que la regla cuyo antecedente es 0-2 (inv-nodes) es aquel que nos presenta una mayor confianza e igual que anteriormente cuando el consecuente es no (node-caps) tenemos las dos reglas de mayor confianza pero sin embargo, cuando es el consecuente, son las dos que menos confianza poseen.

##### Soporte = 0.4; Umbral = 0.5

En este último caso, se genera como máximo valor de k-itemself k=3. Al bajar el valor del soporte mínimo, veos como han aparecido mas reglas que en los apartados anteriores, 22 reglas. Podemos observar en el gráfico de barras que pr primera vez aparecen reglas cuyo lift es menor que 1 estas reglas son: no,left y left,no (node-caps y breast). Cando node-caps = no, es el item que genera un mayor número de reglas, y al igual que ocurría en los apartados anteriores, cambia radicalmente dependienddo de si se trata de antecedente (teniendo los valores de confianza más bajos) o consecuente (dando como resultado los valores más altos).